<a href="https://colab.research.google.com/github/steffenschneider/MRI-tumor-detection/blob/main/mri_tumor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print(123)

123
